In [1]:
import numpy as np
import pandas as pd

In [2]:
from collections import defaultdict
from collections import Counter

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import matplotlib

In [5]:
import pickle

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, roc_auc_score

In [8]:
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split

In [9]:
import pyBigWig

# Load feature matrix

In [2]:
test_MPRA_E116_cell_sp_all = pd.read_csv('test_MPRA_E116_features_other_scores.txt',sep='\t')

In [3]:
test_MPRA_E118_cell_sp_all = pd.read_csv('test_MPRA_E118_features_other_scores.txt',sep='\t')

In [4]:
test_MPRA_E123_cell_sp_all = pd.read_csv('test_MPRA_E123_features_other_scores.txt',sep='\t')

# CV general * cell sp, 1000 replicates

In [5]:
features_generic = ['CHIP','DNASE','PWM','FOOTPRINT','EQTL_2','PWM_matched','FOOTPRINT_matched','IC','IC_matched','funsig','ChIP_var','ChIP_max','ChIP_quantile1','ChIP_quantile2','ChIP_quantile3']

## E116

In [61]:
features_cell_sp = ['H2AFZ_cellSp',
 'H3K27ac_cellSp',
 'H3K27me3_cellSp',
 'H3K36me3_cellSp',
 'H3K4me1_cellSp',
 'H3K4me2_cellSp',
 'H3K4me3_cellSp',
 'H3K79me2_cellSp',
 'H3K9ac_cellSp',
 'H4K20me1_cellSp',
 'DNASE_cellSp',
 'FOOTPRINT_cellSp']

In [77]:
x_1 = np.array(test_MPRA_E116_cell_sp_all[features_generic])
x_2 = np.array(test_MPRA_E116_cell_sp_all[features_cell_sp])
y = test_MPRA_E116_cell_sp_all['Label']

In [78]:
#1000 replicates
fold = 5 
clf_1=RandomForestClassifier(n_estimators=100,random_state=100,oob_score=True)
clf_2=RandomForestClassifier(n_estimators=100,random_state=100,oob_score=True)

In [79]:
roc_aucs = []
ave_prs = []
cors = []
for i in range(200):
    cv = StratifiedKFold(n_splits=fold,shuffle=True)
    for train, test in cv.split(x_1,y):
        y_1 = clf_1.fit(x_1[train], y[train]).predict_proba(x_1[test])[:,1]
        y_2 = clf_2.fit(x_2[train], y[train]).predict_proba(x_2[test])[:,1]
        y_true = y[test]
        roc_auc = roc_auc_score(y_true, y_1 * y_2)
        roc_aucs.append(roc_auc)
        ave_pr = average_precision_score(y_true, y_1 * y_2)
        ave_prs.append(ave_pr)
        cors.append(np.corrcoef(y_1 * y_2,y_true)[0,1])

In [80]:
print (sum(roc_aucs)/1000,sum(ave_prs)/1000,sum(cors)/1000)

0.7314763948274958 0.5482792745100853 0.44960114087791314


In [146]:
x_1 = np.array(test_MPRA_E116_cell_sp_all[features_generic])
y = test_MPRA_E116_cell_sp_all['Label']

In [147]:
roc_aucs = []
ave_prs = []
cors = []
for i in range(200):
    cv = StratifiedKFold(n_splits=fold,shuffle=True)
    for train, test in cv.split(x_1,y):
        y_1 = clf_1.fit(x_1[train], y[train]).predict_proba(x_1[test])[:,1]
        y_true = y[test]
        roc_auc = roc_auc_score(y_true, y_1)
        roc_aucs.append(roc_auc)
        ave_pr = average_precision_score(y_true, y_1)
        ave_prs.append(ave_pr)
        cors.append(np.corrcoef(y_1,y_true)[0,1])

In [148]:
print (sum(roc_aucs)/1000,sum(ave_prs)/1000,sum(cors)/1000)

0.7395770587282814 0.5356147133381409 0.4468235296945204


### clf from ASB cell sp

In [253]:
train_ASB_IC = pickle.load(open('/data/data_repo/shengchd/regulomedb_new_query/1000G_phase3/train_ASB_control_added.pkl','rb'))

In [254]:
features_generic = ['CHIP','DNASE','PWM','FOOTPRINT','EQTL_2','PWM_matched','FOOTPRINT_matched','IC','IC_matched','funsig','ChIP_var','ChIP_max','ChIP_quantile1','ChIP_quantile2','ChIP_quantile3']

In [255]:
x=np.array(train_ASB_IC[features_generic])
y=np.array(train_ASB_IC['label'])
clf=RandomForestClassifier(n_estimators=100,random_state=100,oob_score=True)
clf.fit(x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=True, random_state=100, verbose=0, warm_start=False)

In [256]:
clf.oob_score_

0.788107090004036

In [257]:
pickle.dump(clf,open('/data/data_repo/shengchd/regulomedb_new_query/final_version/ASB_rf_IC_sum_add_control.sav','wb'))

In [258]:
clf_generic = pickle.load(open('/data/data_repo/shengchd/regulomedb_new_query/final_version/ASB_rf_IC_sum_add_control.sav','rb'))

In [259]:
clf = pickle.load(open('/data/data_repo/shengchd/regulomedb_new_query/final_version/ASB_cell_sp_clf_dict.pkl','rb'))

In [260]:
features_cell_sp_ensemble = ['H3K27ac_cellSp',
 'H3K36me3_cellSp',
 'H3K4me1_cellSp',
 'H3K4me3_cellSp',
 'H3K27me3_cellSp',
 'DNASE_cellSp',
 'FOOTPRINT_cellSp']

In [261]:
x_1 = np.array(test_MPRA_E116_cell_sp_all[features_generic])
y_1 = clf_generic.predict_proba(test_MPRA_E116_cell_sp_all[features_generic])[:,1]
x_2 = np.array(test_MPRA_E116_cell_sp_all[features_cell_sp_ensemble])
y_2 = np.mean(np.array([list(clf[c].predict_proba(x_2)[:,1]) for c in clf.keys()]),axis=0)
y = test_MPRA_E116_cell_sp_all['Label']

In [263]:
print (np.round(average_precision_score(y, y_1*y_2),3))
print (np.round(roc_auc_score(y, y_1*y_2),3))
print (np.round(np.corrcoef(y, y_1*y_2)[0,1],3))

0.484
0.724
0.393


## E118

In [88]:
features_cell_sp_2 = ['H3K27ac_cellSp',
 'H3K27me3_cellSp',
 'H3K36me3_cellSp',
 'H3K4me1_cellSp',
 'H3K4me2_cellSp',
 'H3K4me3_cellSp',
 'H3K9ac_cellSp',
 'H3K9me3_cellSp',
 'H4K20me1_cellSp',
 'DNASE_cellSp',
 'FOOTPRINT_cellSp']

In [89]:
x_1 = np.array(test_MPRA_E118_cell_sp_all[features_generic])
x_2 = np.array(test_MPRA_E118_cell_sp_all[features_cell_sp_2])
y = test_MPRA_E118_cell_sp_all['Label']

In [90]:
#1000 replicates
fold = 5 
clf_1=RandomForestClassifier(n_estimators=100,random_state=100,oob_score=True)
clf_2=RandomForestClassifier(n_estimators=100,random_state=100,oob_score=True)

In [ ]:
roc_aucs = []
ave_prs = []
cors = []
for i in range(200):
    print (i)
    cv = StratifiedKFold(n_splits=fold,shuffle=True)
    for train, test in cv.split(x_1,y):
        y_1 = clf_1.fit(x_1[train], y[train]).predict_proba(x_1[test])[:,1]
        y_2 = clf_2.fit(x_2[train], y[train]).predict_proba(x_2[test])[:,1]
        y_true = y[test]
        roc_auc = roc_auc_score(y_true, y_1 * y_2)
        roc_aucs.append(roc_auc)
        ave_pr = average_precision_score(y_true, y_1 * y_2)
        ave_prs.append(ave_pr)
        cors.append(np.corrcoef(y_1 * y_2,y_true)[0,1])

In [92]:
print (sum(roc_aucs)/1000,sum(ave_prs)/1000,sum(cors)/1000)

0.7654597273097272 0.5705757443960903 0.4226768752305934


In [149]:
x_1 = np.array(test_MPRA_E118_cell_sp_all[features_generic])
y = test_MPRA_E118_cell_sp_all['Label']

In [150]:
roc_aucs = []
ave_prs = []
cors = []
for i in range(200):
    cv = StratifiedKFold(n_splits=fold,shuffle=True)
    for train, test in cv.split(x_1,y):
        y_1 = clf_1.fit(x_1[train], y[train]).predict_proba(x_1[test])[:,1]
        y_true = y[test]
        roc_auc = roc_auc_score(y_true, y_1)
        roc_aucs.append(roc_auc)
        ave_pr = average_precision_score(y_true, y_1)
        ave_prs.append(ave_pr)
        cors.append(np.corrcoef(y_1,y_true)[0,1])

In [151]:
print (sum(roc_aucs)/1000,sum(ave_prs)/1000,sum(cors)/1000)

0.733480862667142 0.5018751361229498 0.3717931578433827


### clf from ASB cell sp

In [264]:
x_1 = np.array(test_MPRA_E118_cell_sp_all[features_generic])
y_1 = clf_generic.predict_proba(test_MPRA_E118_cell_sp_all[features_generic])[:,1]
x_2 = np.array(test_MPRA_E118_cell_sp_all[features_cell_sp_ensemble])
y_2 = np.mean(np.array([list(clf[c].predict_proba(x_2)[:,1]) for c in clf.keys()]),axis=0)
y = test_MPRA_E118_cell_sp_all['Label']

In [265]:
print (np.round(average_precision_score(y, y_1*y_2),3))
print (np.round(roc_auc_score(y, y_1*y_2),3))
print (np.round(np.corrcoef(y, y_1*y_2)[0,1],3))

0.479
0.728
0.357


## E123

In [98]:
features_cell_sp_3 = ['H2AFZ_cellSp',
 'H3K27ac_cellSp',
 'H3K27me3_cellSp',
 'H3K36me3_cellSp',
 'H3K4me1_cellSp',
 'H3K4me2_cellSp',
 'H3K4me3_cellSp',
 'H3K79me2_cellSp',
 'H3K9ac_cellSp',
 'H3K9me3_cellSp',
 'H4K20me1_cellSp',
 'DNASE_cellSp',
 'FOOTPRINT_cellSp']

In [99]:
x_1 = np.array(test_MPRA_E123_cell_sp_all[features_generic])
x_2 = np.array(test_MPRA_E123_cell_sp_all[features_cell_sp_3])
y = test_MPRA_E123_cell_sp_all['Label']

In [100]:
#1000 replicates
fold = 5 
clf_1=RandomForestClassifier(n_estimators=100,random_state=100,oob_score=True)
clf_2=RandomForestClassifier(n_estimators=100,random_state=100,oob_score=True)

In [ ]:
roc_aucs = []
ave_prs = []
cors = []
for i in range(200):
    print (i)
    cv = StratifiedKFold(n_splits=fold,shuffle=True)
    for train, test in cv.split(x_1,y):
        y_1 = clf_1.fit(x_1[train], y[train]).predict_proba(x_1[test])[:,1]
        y_2 = clf_2.fit(x_2[train], y[train]).predict_proba(x_2[test])[:,1]
        y_true = y[test]
        roc_auc = roc_auc_score(y_true, y_1 * y_2)
        roc_aucs.append(roc_auc)
        ave_pr = average_precision_score(y_true, y_1 * y_2)
        ave_prs.append(ave_pr)
        cors.append(np.corrcoef(y_1 * y_2,y_true)[0,1])

In [102]:
print (sum(roc_aucs)/1000,sum(ave_prs)/1000,sum(cors)/1000)

0.6809207977509142 0.44123371359509495 0.34361420281256727


In [152]:
x_1 = np.array(test_MPRA_E123_cell_sp_all[features_generic])
y = test_MPRA_E123_cell_sp_all['Label']

In [153]:
roc_aucs = []
ave_prs = []
cors = []
for i in range(200):
    cv = StratifiedKFold(n_splits=fold,shuffle=True)
    for train, test in cv.split(x_1,y):
        y_1 = clf_1.fit(x_1[train], y[train]).predict_proba(x_1[test])[:,1]
        y_true = y[test]
        roc_auc = roc_auc_score(y_true, y_1)
        roc_aucs.append(roc_auc)
        ave_pr = average_precision_score(y_true, y_1)
        ave_prs.append(ave_pr)
        cors.append(np.corrcoef(y_1,y_true)[0,1])

In [154]:
print (sum(roc_aucs)/1000,sum(ave_prs)/1000,sum(cors)/1000)

0.6640223261535239 0.3998973942002876 0.29382344752671047


### clf from ASB cell sp

In [266]:
x_1 = np.array(test_MPRA_E123_cell_sp_all[features_generic])
y_1 = clf_generic.predict_proba(test_MPRA_E123_cell_sp_all[features_generic])[:,1]
x_2 = np.array(test_MPRA_E123_cell_sp_all[features_cell_sp_ensemble])
y_2 = np.mean(np.array([list(clf[c].predict_proba(x_2)[:,1]) for c in clf.keys()]),axis=0)
y = test_MPRA_E123_cell_sp_all['Label']

In [267]:
print (np.round(average_precision_score(y, y_1*y_2),3))
print (np.round(roc_auc_score(y, y_1*y_2),3))
print (np.round(np.corrcoef(y, y_1*y_2)[0,1],3))

0.329
0.676
0.253
